Question 1: Please help me find out the license thing

### first approach 

In [ ]:
%time
# PDF to HTML
# http://macappstore.org/pdf2htmlex/
# https://github.com/coolwanglu/pdf2htmlEX/wiki/Command-Line-Options
import os
echo = "pdf2htmlEX --zoom 1.3"
for filename in os.listdir("pdf_files"):
    if filename.endswith("pdf"):
        print(filename)
        #os.system(echo+" "+filename)

## second approach (current one)

In [ ]:
%time
# PDF to HTML
# https://sourceforge.net/projects/pdftohtml/
# https://www.experts-exchange.com/videos/59892/Xpdf-PDFtoHTML-Command-Line-Utility-to-Convert-a-PDF-File-to-HTML.html
# https://www.xpdfreader.com/download.html

# make sure to have a folder named "pdf_files" with the pdfs
# make sure to have an empty folder named "html_files"
import os
echo = "pdftohtml"
for filename in os.listdir("pdf_files"):
    if filename.endswith("pdf"):
        print(filename)
        file_pdf = "pdf_files/" + filename
        file_html = "html_files/" + filename
        #os.system(echo+" "+file_pdf+" "+file_html) 

## creates table of features

In [1]:
import codecs
from bs4 import BeautifulSoup
import cssutils
import pandas as pd
import os

In [2]:
def style_dict(style_tem):
    style_tem = style_tem[:-1]
    style_dic = dict(item.split(":") for item in style_tem.split(";"))
    return style_dic

In [3]:
def css_style(soup):
    selectors = {}
    for styles in soup.select('style'):
        css = cssutils.parseString(styles.encode_contents())
        for rule in css:
            if rule.type == rule.STYLE_RULE:
                style = rule.selectorText
                selectors[style] = {}
                for item in rule.style:
                    propertyname = item.name
                    value = item.value
                    selectors[style][propertyname] = value

    dict_css = {key.strip('#'): item for key, item in selectors.items()}
    return dict_css

In [17]:
def extract_features(path):
    with open(path) as raw_resuls:
        results = BeautifulSoup(raw_resuls, 'lxml')
    
    dict_css = css_style(results)
    lines = []
    text = []
    for element in results.find_all("body"):
        for stat in element.find_all("div"):
            data={}
            # gets the info from <div>
            div_dic = stat.attrs
            data.update(div_dic)

            # gets info from <span>
            span_dic = stat.find('span').attrs
            data.update(span_dic)

            # gets the features from style in <div>
            style_ = span_dic.get('style')
            style_span = style_dict(style_)
            data.update(style_span)

            # gets features of the font from css 
            font = span_dic.get('id')
            temp = dict_css.get(font)
            data.update(temp)

            # gets the features from style in <div>
            style_ = div_dic.get('style')
            style_temp = style_dict(style_)
            data.update(style_temp)
            
            d = pd.DataFrame.from_dict(data)
            
            text.append(stat.text)
            #print(stat.text)
            lines.append(d)

    dataframe = pd.concat(lines, axis=0).reset_index(drop=1)
    dataframe["text"] = pd.Series( (v for v in text))
    print(dataframe.columns)
    
    return dataframe

### reads all html files

In [18]:
%time
features = []
for root, dirs, files in os.walk("html_files"):
    for file in files:
        if file.endswith(".html"):
            if file != "index.html":
                try:
                    file_path =  os.path.join(root, file)
                    print(file_path+ " found")
                    df = extract_features(file_path)
                    df = df.assign(page=file)
                    df = df.assign(document=root)
                    features.append(df)
                except:
                    print(file_path+" NOT found")

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 6.91 µs
html_files/CMS_2014_0115_0059.pdf/page1.html found
Index(['class', 'style', 'id', 'font-size', 'vertical-align', 'color',
       'font-family', 'font-weight', 'font-style', 'position', ' left', ' top',
       'text'],
      dtype='object')
html_files/CMS_2014_0115_0059.pdf/page2.html found
Index(['class', 'style', 'id', 'font-size', 'vertical-align', 'color',
       'font-family', 'font-weight', 'font-style', 'position', ' left', ' top',
       'text'],
      dtype='object')
html_files/CMS_2014_0115_0059.pdf/page3.html found
Index(['class', 'style', 'id', 'font-size', 'vertical-align', 'color',
       'font-family', 'font-weight', 'font-style', 'position', ' left', ' top',
       'text'],
      dtype='object')
html_files/CMS_2014_0115_0059.pdf/page4.html found
Index(['class', 'style', 'id', 'font-size', 'vertical-align', 'color',
       'font-family', 'font-weight', 'font-style', 'position', ' left', ' top',
       'text']

Index(['class', 'style', 'id', 'font-size', 'vertical-align', 'color',
       'font-family', 'font-weight', 'font-style', 'position', ' left', ' top',
       'text'],
      dtype='object')
html_files/CMS_2014_0115_12304.pdf/page2.html found
Index(['class', 'style', 'id', 'font-size', 'vertical-align', 'color',
       'font-family', 'font-weight', 'font-style', 'position', ' left', ' top',
       'text'],
      dtype='object')
html_files/CMS_2014_0115_12304.pdf/page3.html found
Index(['class', 'style', 'id', 'font-size', 'vertical-align', 'color',
       'font-family', 'font-weight', 'font-style', 'position', ' left', ' top',
       'text'],
      dtype='object')
html_files/CMS_2014_0115_12304.pdf/page4.html found
Index(['class', 'style', 'id', 'font-size', 'vertical-align', 'color',
       'font-family', 'font-weight', 'font-style', 'position', ' left', ' top',
       'text'],
      dtype='object')
html_files/CMS_2014_0115_12304.pdf/page5.html found
Index(['class', 'style', 'id', 'font

Index(['class', 'style', 'id', 'font-size', 'vertical-align', 'color',
       'font-family', 'font-weight', 'font-style', 'position', ' left', ' top',
       'text'],
      dtype='object')
html_files/CMS_2014_0115_12762.pdf/page3.html found
Index(['class', 'style', 'id', 'font-size', 'vertical-align', 'color',
       'font-family', 'font-weight', 'font-style', 'position', ' left', ' top',
       'text'],
      dtype='object')
html_files/CMS_2014_0115_12762.pdf/page4.html found
Index(['class', 'style', 'id', 'font-size', 'vertical-align', 'color',
       'font-family', 'font-weight', 'font-style', 'position', ' left', ' top',
       'text'],
      dtype='object')
html_files/CMS_2014_0115_12762.pdf/page5.html found
Index(['class', 'style', 'id', 'font-size', 'vertical-align', 'color',
       'font-family', 'font-weight', 'font-style', 'position', ' left', ' top',
       'text'],
      dtype='object')
html_files/CMS_2014_0115_12762.pdf/page6.html found
Index(['class', 'style', 'id', 'font

Index(['class', 'style', 'id', 'font-size', 'vertical-align', 'color',
       'font-family', 'font-weight', 'font-style', 'position', ' left', ' top',
       'text'],
      dtype='object')
html_files/CMS_2014_0115_13028.pdf/page1.html found
Index(['class', 'style', 'id', 'font-size', 'vertical-align', 'color',
       'font-family', 'font-weight', 'font-style', 'position', ' left', ' top',
       'text'],
      dtype='object')
html_files/CMS_2014_0115_13028.pdf/page2.html found
Index(['class', 'style', 'id', 'font-size', 'vertical-align', 'color',
       'font-family', 'font-weight', 'font-style', 'position', ' left', ' top',
       'text'],
      dtype='object')
html_files/CMS_2014_0115_13028.pdf/page3.html found
Index(['class', 'style', 'id', 'font-size', 'vertical-align', 'color',
       'font-family', 'font-weight', 'font-style', 'position', ' left', ' top',
       'text'],
      dtype='object')
html_files/CMS_2014_0115_13028.pdf/page4.html found
Index(['class', 'style', 'id', 'font

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:40: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



...

Index(['class', 'style', 'id', 'font-size', 'vertical-align', 'color',
       'font-family', 'font-weight', 'font-style', 'position', ' left', ' top',
       'text'],
      dtype='object')
html_files/CMS_2014_0115_58260.pdf/page5.html found
Index(['class', 'style', 'id', 'font-size', 'vertical-align', 'color',
       'font-family', 'font-weight', 'font-style', 'position', ' left', ' top',
       'text'],
      dtype='object')
html_files/CMS_2014_0115_58260.pdf/page6.html found
Index(['class', 'style', 'id', 'font-size', 'vertical-align', 'color',
       'font-family', 'font-weight', 'font-style', 'position', ' left', ' top',
       'text'],
      dtype='object')
html_files/CMS_2014_0115_58260.pdf/page7.html found
Index(['class', 'style', 'id', 'font-size', 'vertical-align', 'color',
       'font-family', 'font-weight', 'font-style', 'position', ' left', ' top',
       'text'],
      dtype='object')
html_files/CMS_2014_0115_58260.pdf/page8.html found
Index(['class', 'style', 'id', 'font

Index(['class', 'style', 'id', 'font-size', 'vertical-align', 'color',
       'font-family', 'font-weight', 'font-style', 'position', ' left', ' top',
       'text'],
      dtype='object')
html_files/CMS_2014_0115_58375.pdf/page14.html found
Index(['class', 'style', 'id', 'font-size', 'vertical-align', 'color',
       'font-family', 'font-weight', 'font-style', 'position', ' left', ' top',
       'text'],
      dtype='object')
html_files/CMS_2014_0115_58375.pdf/page15.html found
Index(['class', 'style', 'id', 'font-size', 'vertical-align', 'color',
       'font-family', 'font-weight', 'font-style', 'position', ' left', ' top',
       'text'],
      dtype='object')
html_files/CMS_2014_0115_58375.pdf/page2.html found
Index(['class', 'style', 'id', 'font-size', 'vertical-align', 'color',
       'font-family', 'font-weight', 'font-style', 'position', ' left', ' top',
       'text'],
      dtype='object')
html_files/CMS_2014_0115_58375.pdf/page3.html found
Index(['class', 'style', 'id', 'fo

Index(['class', 'style', 'id', 'font-size', 'vertical-align', 'color',
       'font-family', 'font-weight', 'font-style', 'position', ' left', ' top',
       'text'],
      dtype='object')
html_files/CMS_2014_0115_58432.pdf/page2.html found
Index(['class', 'style', 'id', 'font-size', 'vertical-align', 'color',
       'font-family', 'font-weight', 'font-style', 'position', ' left', ' top',
       'text'],
      dtype='object')
html_files/CMS_2014_0115_58432.pdf/page3.html found
Index(['class', 'style', 'id', 'font-size', 'vertical-align', 'color',
       'font-family', 'font-weight', 'font-style', 'position', ' left', ' top',
       'text'],
      dtype='object')
html_files/CMS_2014_0115_58432.pdf/page4.html found
Index(['class', 'style', 'id', 'font-size', 'vertical-align', 'color',
       'font-family', 'font-weight', 'font-style', 'position', ' left', ' top',
       'text'],
      dtype='object')
html_files/CMS_2014_0115_58432.pdf/page5.html found
Index(['class', 'style', 'id', 'font

Index(['class', 'style', 'id', 'font-size', 'vertical-align', 'color',
       'font-family', 'font-weight', 'font-style', 'position', ' left', ' top',
       'text'],
      dtype='object')
html_files/CMS_2014_0115_58438.pdf/page7.html found
Index(['class', 'style', 'id', 'font-size', 'vertical-align', 'color',
       'font-family', 'font-weight', 'font-style', 'position', ' left', ' top',
       'text'],
      dtype='object')
html_files/CMS_2014_0115_58439.pdf/page1.html found
Index(['class', 'style', 'id', 'font-size', 'vertical-align', 'color',
       'font-family', 'font-weight', 'font-style', 'position', ' left', ' top',
       'text'],
      dtype='object')
html_files/CMS_2014_0115_58439.pdf/page2.html found
Index(['class', 'style', 'id', 'font-size', 'vertical-align', 'color',
       'font-family', 'font-weight', 'font-style', 'position', ' left', ' top',
       'text'],
      dtype='object')
html_files/CMS_2014_0115_58439.pdf/page3.html found
Index(['class', 'style', 'id', 'font

Index(['class', 'style', 'id', 'font-size', 'vertical-align', 'color',
       'font-family', 'font-weight', 'font-style', 'position', ' left', ' top',
       'text'],
      dtype='object')
html_files/CMS_2014_0115_9664.pdf/page2.html found
Index(['class', 'style', 'id', 'font-size', 'vertical-align', 'color',
       'font-family', 'font-weight', 'font-style', 'position', ' left', ' top',
       'text'],
      dtype='object')
html_files/CMS_2014_0115_9664.pdf/page3.html found
Index(['class', 'style', 'id', 'font-size', 'vertical-align', 'color',
       'font-family', 'font-weight', 'font-style', 'position', ' left', ' top',
       'text'],
      dtype='object')
html_files/CMS_2014_0115_9664.pdf/page4.html found
Index(['class', 'style', 'id', 'font-size', 'vertical-align', 'color',
       'font-family', 'font-weight', 'font-style', 'position', ' left', ' top',
       'text'],
      dtype='object')
html_files/CMS_2014_0115_9981.pdf/page1.html found
Index(['class', 'style', 'id', 'font-siz

In [19]:
a = pd.concat(features, axis=0, sort=False).reset_index(drop=1)

In [20]:
a

,class,style,id,font-size,vertical-align,color,font-family,font-weight,font-style,position,left,top,text,page,document,line-height
0,txt,font-size:11px;vertical-align:baseline;color:r...,f2,11px,baseline,"rgba(0,0,0,1)",sans-serif,normal,normal,absolute,72px,154px,"October 10, 2014",page1.html,html_files/CMS_2014_0115_0059.pdf,NaN
1,txt,font-size:11px;vertical-align:baseline;color:r...,f2,11px,baseline,"rgba(0,0,0,1)",sans-serif,normal,normal,absolute,72px,181px,Centers for Medicare & Medicaid Services,page1.html,html_files/CMS_2014_0115_0059.pdf,NaN
2,txt,font-size:11px;vertical-align:baseline;color:r...,f2,11px,baseline,"rgba(0,0,0,1)",sans-serif,normal,normal,absolute,72px,194px,Department of Health and Human Services,page1.html,html_files/CMS_2014_0115_0059.pdf,NaN
3,txt,font-size:11px;vertical-align:baseline;color:r...,f2,11px,baseline,"rgba(0,0,0,1)",sans-serif,normal,normal,absolute,72px,208px,Attention: CMS-9968-P,page1.html,html_files/CMS_2014_0115_0059.pdf,NaN
4,txt,font-size:11px;vertical-align:baseline;color:r...,f2,11px,baseline,"rgba(0,0,0,1)",sans-serif,normal,normal,absolute,72px,221px,P.O. Box 8013,page1.html,html_files/CMS_2014_0115_0059.pdf,NaN
5,txt,font-size:11px;vertical-align:baseline;color:r...,f2,11px,baseline,"rgba(0,0,0,1)",sans-serif,normal,normal,absolute,72px,235px,"Baltimore, MD 21244-1850",page1.html,html_files/CMS_2014_0115_0059.pdf,NaN
6,txt,font-size:11px;vertical-align:baseline;color:r...,f2,11px,baseline,"rgba(0,0,0,1)",sans-serif,normal,normal,absolute,72px,262px,Re: Nationwide Life Insurance Company’s commen...,page1.html,html_files/CMS_2014_0115_0059.pdf,NaN
7,txt,font-size:11px;vertical-align:baseline;color:r...,f2,11px,baseline,"rgba(0,0,0,1)",sans-serif,normal,normal,absolute,72px,275px,students of religious institutions,page1.html,html_files/CMS_2014_0115_0059.pdf,NaN
8,txt,font-size:11px;vertical-align:baseline;color:r...,f2,11px,baseline,"rgba(0,0,0,1)",sans-serif,normal,normal,absolute,72px,302px,To Whom It May Concern:,page1.html,html_files/CMS_2014_0115_0059.pdf,NaN
9,txt,font-size:12px;vertical-align:baseline;color:r...,f3,12px,baseline,"rgba(0,0,0,1)",sans-serif,normal,normal,absolute,72px,329px,On behalf of Nationwide Life Insurance Company...,page1.html,html_files/CMS_2014_0115_0059.pdf,NaN
